## 경사하강을 이용한 행렬 분해
P와 Q 행렬로 계산된 예측 행렬 R 값이 실제 R 행렬값과 가장 최소의 오류를 가질 수 있도록 반복적인 비용함수 최적화를 통해 P와 Q를 유추해내는 것입니다. 

1. P와 Q를 임의의 값을 가진 행렬로 설정합니다.
2. P와 Q.T값을 곱해 예측 R 행렬을 계산하고 예측 R 행렬과 실제 R 행렬에 해당하는 오류값을 계산합니다.
3. 이 오류값을 최소화할 수 있도록 P와 Q 행렬을 적절한 값으로 각각 업데이트 합니다.
4. 만족할만한 오류값을 가질 때까지 2, 3번 작업을 반복하면서 P와 Q 값을 업데이트해 근사화합니다.

In [1]:
import numpy as np

# 원본행렬 R 생성, 분해행렬 P와 Q 초기화, 잠재요인 차원 K는 3 설정
R = np.array([[4, np.NaN, np.NaN, 2, np.NaN],
              [np.NaN, 5, np.NaN, 3, 1],
              [np.NaN, np.NaN, 3, 4, 4],
              [5, 2, 1, 2, np.NaN]])
num_users, num_items = R.shape
K = 3

# P와 Q 매트릭스의 크기를 지정하고 정규분포를 가진 random 한 값으로 입력합니다.
np.random.seed(1)
P = np.random.normal(scale=1./K, size=(num_users, K))
Q = np.random.normal(scale=1./K, size=(num_items, K))

In [5]:
from sklearn.metrics import mean_squared_error

def get_rmse(R, P, Q, non_zeros):
    error = 0
    # 두개의 분해된 행렬 P, Q.T의 내적으로 예측 행렬 R행렬 생성
    full_pred_matrix = np.dot(P, Q.T)

    # 실제 R 행렬에서 널이 아닌 값의 위치 인덱스 추출하여 실제 R 행렬과 예측 행렬의 RMSE 추출
    x_non_zero_ind = [non_zero[0] for non_zero in non_zeros]
    y_non_zero_ind = [non_zero[1] for non_zero in non_zeros]
    R_non_zeros = R[x_non_zero_ind, y_non_zero_ind]
    full_pred_matrix_non_zeros = full_pred_matrix[x_non_zero_ind, y_non_zero_ind]

    mse = mean_squared_error(R_non_zeros, full_pred_matrix_non_zeros)
    rmse = np.sqrt(mse)

    return rmse

In [6]:
# R > 0 인 행 위치, 열위치, 값을 non_zero 리스트에 저장
non_zeros = [(i, j, R[i, j]) for i in range(num_users) for j in range(num_items) if R[i, j] > 0]

steps = 1000
learning_rate = 0.01
r_lambda=0.01

# SGD 기법으로 P와 Q 매트릭스를 계속 업데이트
for step in range(steps):
    for i, j, r in non_zeros:
        # 실제값과 예측값의 차이인 오류 값 구함
        eij = r - np.dot(P[i, :], Q[j, :].T)
        # Regularization을 반영한 SGD 업데이트 공식 적용
        P[i,:] = P[i,:] + learning_rate*(eij * Q[j, :] - r_lambda*P[i, :])
        Q[j,:] = Q[j,:] + learning_rate*(eij * P[i, :] - r_lambda*Q[j,:])

    rmse = get_rmse(R, P, Q, non_zeros)
    if (step % 50) == 0:
        print("### iteration step : ", step, " rmse : ", rmse)

### iteration step :  0  rmse :  3.2142352850266604
### iteration step :  50  rmse :  0.47024902717741013
### iteration step :  100  rmse :  0.15376535936658764
### iteration step :  150  rmse :  0.07361687046634227
### iteration step :  200  rmse :  0.042854422724033434
### iteration step :  250  rmse :  0.02906178882983784
### iteration step :  300  rmse :  0.02253267267313507
### iteration step :  350  rmse :  0.019452104360723494
### iteration step :  400  rmse :  0.018003062614111706
### iteration step :  450  rmse :  0.01731014561278133
### iteration step :  500  rmse :  0.016968868634676948
### iteration step :  550  rmse :  0.016794291226021912
### iteration step :  600  rmse :  0.01669990557944416
### iteration step :  650  rmse :  0.016643834424762662
### iteration step :  700  rmse :  0.016605230293015815
### iteration step :  750  rmse :  0.01657359666938704
### iteration step :  800  rmse :  0.016543716472882376
### iteration step :  850  rmse :  0.016513124797746925
### i

In [7]:
pred_matrix = np.dot(P, Q.T)
print('예측행렬:\n', np.round(pred_matrix, 3))

예측행렬:
 [[3.991 0.897 1.306 2.002 1.663]
 [6.695 4.978 0.978 2.981 1.003]
 [6.677 0.391 2.987 3.977 3.986]
 [4.968 2.005 1.006 2.017 1.141]]
